# Salty Boussinesq Melting

This notebook will perform the analysis of the salty Boussinesq melting simulations.

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as de

d = de.operators.differentiate
integ = de.operators.integrate

import glob
from dedalus.tools import post
import file_tools as flt
import interpolation as ip
import time
import os
import logging
import sys
logger = logging.getLogger(__name__)

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

savedir = './salty-boussinesq-melting'

# Coordinate remapping


In this notebook, we use a coordinate remapping to simulate flow underneath an ice sheet with topography.

The coordinate system is defined as follows.

Cartesian coordinates are $t, x, z$.

Remapped coordinates and $\tau, \xi, \zeta$, which are defined as

\begin{align}
\tau &= \frac{t}{\mathcal{T}} &
\xi &= \frac{x}{\mathcal{L}} &
\zeta &= \frac{z}{\mathcal{L}\eta(\tau,\xi)}
\end{align}

This lets us write all the partial derivatives with respect to each other

\begin{align}
\frac{\partial {\xi}_i}{\partial{x}_j} &= 
    \begin{bmatrix}
    \partial_t \tau & \partial_t \xi & \partial_t \zeta \\
    \partial_x \tau & \partial_x \xi & \partial_x \zeta \\
    \partial_z \tau & \partial_z \xi & \partial_z \zeta \\
    \end{bmatrix}
    = \begin{bmatrix}
    \frac{1}{\mathcal{T}} & 0 & -\frac{1}{\mathcal{T}} \zeta \frac{\partial_\tau \eta}{\eta}\\
    0 & \frac{1}{\mathcal{L}} & -\frac{1}{\mathcal{L}} \zeta \frac{\partial_\xi \eta}{\eta}\\
    0 & 0 & \frac{1}{\mathcal{L} \eta}
    \end{bmatrix}\\
\frac{\partial {x}_i}{\partial{\xi}_j} &= 
    \begin{bmatrix}
    \partial_\tau t & \partial_\tau x & \partial_\tau z \\
    \partial_\xi t & \partial_\xi x & \partial_\xi z \\
    \partial_\zeta t & \partial_\zeta x & \partial_\zeta z \\
    \end{bmatrix} 
    = \begin{bmatrix}
    \mathcal{T} & 0 & \mathcal{L} \zeta \partial_\tau \eta\\
    0 & \mathcal{L} & \mathcal{L} \zeta \partial_\xi \eta\\
    0 & 0 & \mathcal{L} \eta
    \end{bmatrix}
\end{align}

We will determine the differential geometric quantities in the spatial directions, and consider time derivatives separately (as a metric makes sense spatially only).
Remember, everything here will in general also vary in time

We get the **tangent vectors**

\begin{align}
e_1 = \frac{\partial\bf{x}}{\partial\xi} \cong \frac{\partial}{\partial\xi} &= \mathcal{L} \hat{e}_1 + \zeta \partial_\xi \eta  \, \mathcal{L}\hat{e}_2\\
e_2 = \frac{\partial\bf{x}}{\partial\zeta} \cong \frac{\partial}{\partial\zeta} &=\eta \, \mathcal{L} \hat{e}_2
\end{align}

The have corresponding **dual/covectors**

\begin{align}
\omega^1 = \nabla \xi = \bf{d} \xi &= \frac{1}{\mathcal{L}}\hat{\omega}^1 \\
\omega^2 = \nabla \zeta = \bf{d} \zeta &= - \zeta \frac{\partial_\xi \eta}{\eta} \, \frac{1}{\mathcal{L}} \hat{\omega}^1 +  \frac{1}{\eta}\frac{1}{\mathcal{L}} \hat{\omega}^2
\end{align}

We can then calculate the **metric** components

\begin{align}
g_{11} &= \mathcal{L}^2(1 + \zeta^2 \partial_\xi \eta^2)  &
    g^{11} &= \frac{1}{\mathcal{L}^2}\\
g_{12} = g_{21} &= \mathcal{L}^2 \zeta \eta \partial_\xi \eta &
    g^{12} = g^{21} &= - \frac{1}{\mathcal{L}^2} \zeta \frac{\partial_\xi \eta}{\eta} \\
g_{22} &= \mathcal{L}^2 \eta^2 &
    g^{22} &= \frac{1}{\mathcal{L}^2} \frac{1}{\eta^2}\left(1 + \zeta^2 \partial_\xi \eta^2 \right)
\end{align}

This has determinant of
$$ g = \mathcal{L}^4 \eta^2 $$
and **Jacobian**
$$ \sqrt{g} = \mathcal{L}^2 \eta$$

We calculate the **connection coefficients**/Christoffel symbols directly using
\begin{align}
\Gamma^\gamma_{\alpha \beta} = \langle \omega^\gamma, \nabla_\beta e_\alpha\rangle
\end{align}

Hence
\begin{align}
\Gamma^1_{11} &= 0 & \Gamma^2_{11} &= \zeta \frac{\partial^2_\xi \eta}{\eta}\\
\Gamma^1_{12} &= 0 & \Gamma^2_{12} &= \frac{\partial_\xi \eta}{\eta}\\
\Gamma^1_{21} &= 0 & \Gamma^2_{21} &= \frac{\partial_\xi \eta}{\eta}\\
\Gamma^1_{22} &= 0 & \Gamma^2_{22} &= 0
\end{align}
Of course, for coordinate bases we have symmetry in the last two components.

We can write the **divergence** using
$$\nabla \cdot u = \nabla u : I = (\omega^j \nabla_j)(u^i e_i): = (u^i_{,j} + u^j \Gamma^i_{ji})\omega^j\cdot e_i = (u^i_{,i} + u^j \Gamma^i_{ji})$$
$$\nabla \cdot u = u^i_{;i} = u^i_{,i} + \Gamma^i_{j i} u^j = u^1_{,1} + u^2_{,2} + \frac{\partial_\xi \eta}{\eta} u^1$$

We write the **vorticity** (in covariant components) as

$$q = \nabla \times u = \mathcal{E}:\nabla u = (\mathcal{E}^{ij}e_i e_j):(\omega^k \nabla_k)(u^l \, e_l) = \frac{1}{\sqrt{g}} [ij] u^l_{;k}(e_i\cdot\omega^k)(e_j\cdot e_l)$$
$$ = \frac{1}{\sqrt{g}} [ij] g_{jl} u^l_{;i} = \frac{1}{\sqrt{g}} ( g_{2l} u^l_{;1} - g_{1l} u^l_{;2})$$

We can write the **curl of the vorticity** as 

$$ \nabla \times q = \mathcal{E}\cdot\nabla q = (\mathcal{E}^{ij}e_i e_j)\cdot(\omega^k \nabla_k)q =  \frac{1}{\sqrt{g}}[ij]q_{,i} e_j = \frac{1}{\sqrt{g}}(q_{,1}e_2 - q_{,2}e_1) $$

Note that this sign appears to be incorrect.

The **gradient** of the pressure is just

$$\nabla p = g^{ji} p_{,j} e_i$$

The **cross product** $u \times q$ is 

$$u\times q = \mathcal{E}\cdot u q = (\mathcal{E}_{ij}\omega^i\omega^j)\cdot(q u^k e_k) = \sqrt{g}q [ij] \delta^i_k u^k \omega^j = \sqrt{g} g^{jk} [ij] u^i e_k$$
$$ = \sqrt{g}q(g^{2k} u^1 - g^{1k}u^2)e_k$$

**Time derivative**

$$ \partial_t = \partial_t\tau \partial_\tau + \partial_t \xi \partial_\xi + \partial_t \zeta \partial_\zeta = \frac{1}{\mathcal{T}} (\partial_\tau - \zeta \frac{\partial_\tau \eta}{\eta} \partial_\zeta)$$

And 

$$ \partial_t u = \partial_t (u^i e_i) = \partial_t u^i e_i + u^i \partial_t e_i$$

where

$$\partial_\tau e_1 = \mathcal{L} \zeta \partial_\tau \partial_\xi \eta \hat{e}_2 = \frac{\zeta}{\eta}\partial_\tau\partial_\xi \eta\, e_2$$
$$\partial_\tau e_2 = \mathcal{L} \partial_\tau \eta \hat{e}_2 = \frac{\partial_\tau\eta}{\eta}\, e_2$$

## Plotting the coordinate transformation

In [ ]:
# Create Cartesian grid
n = 64
xb = de.Fourier('x',n,interval=(0,4))
zb = de.Chebyshev('z',n,interval=(0,1))
d0 = de.Domain([xb,zb],grid_dtype=np.float64)
xg,zg = d0.grids()
xxg,zzg = xg+0*zg, 0*xg+zg

# Create remapped grid
ξb = de.Fourier('ξ',n,interval=(0,4))
ζb = de.Chebyshev('ζ',n,interval=(0,1))
d1 = de.Domain([ξb,ζb],grid_dtype=np.float64)
ξg,ζg = d1.grids()
ξξg,ζζg = ξg+0*ζg, 0*ξg+ζg
ηg = 1 + .25*(1-np.cos(np.pi*xg/2))

# Create dedalus fields to store various quantities for each domain
f0,f1,f2 = {},{},{}
def create_fields(names,dic,d,dim='x'): 
    for name in names: 
        dic[name] = d.new_field()
        dic[name].meta[dim]['parity'] = 1
create_fields(['x','z','η','ξ','ζ1','ζ2'],f0,d0)
create_fields(['x','z1','z2','η','ξ','ζ'],f1,d1,dim='ξ')
# Quantities on Cartesian grid
f0['x']['g'] = xg
f0['z']['g'] = zg
f0['η']['g'] = ηg
f0['ξ']['g'] = xg
f0['ζ1']['g'] = (zg-ηg)/(2-ηg)
f0['ζ2']['g'] = zg/ηg
# Quantities in remapped space
f1['η']['g'] = ηg
f1['ξ']['g'] = ξg
f1['ζ']['g'] = ζg
f1['x']['g'] = xg
f1['z1']['g'] = ηg + (2-ηg)*ζg
f1['z2']['g'] = ηg*ζg

In [ ]:
# Plot ξ and ζ level sets in Cartesian (x,z) space (at ξ,ζ grid points)
fig, ax = plt.subplots(figsize=(8,4),constrained_layout=True)
cplot1 = ax.contourf(f1['x']['g'],f1['z1']['g'],f1['ζ']['g'],np.linspace(0,1,6),cmap='viridis_r',vmin=0,vmax=1)
cplot2 = ax.contourf(f1['x']['g'],f1['z2']['g'],f1['ζ']['g'],np.linspace(0,1,6),cmap='magma',vmin=-.2,vmax=1)
ax.contour(f1['x']['g'],f1['z1']['g'],f1['ξ']['g'],np.linspace(0,4,17),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z2']['g'],f1['ξ']['g'],np.linspace(0,4,17),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z1']['g'],f1['ζ']['g'],np.linspace(0,1,6),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z2']['g'],f1['ζ']['g'],np.linspace(0,1,6),linewidths=0.5,colors='k',)
ax.plot(ξg,ηg,'k')
for spine in ax.spines: ax.spines[spine].set_visible(False)
ax.set_xlabel('$x$',fontsize=16)
ax.set_ylabel('$z$',fontsize=16)
ax.set_title('$\\zeta^-$ and $\\zeta^+$ in Cartesian coordinates',fontsize=16)
ax.set(aspect=1,xticks=np.linspace(0,4,16,endpoint=False),yticks=np.linspace(0,2,9))
cbax1 = fig.add_axes([0.91, 0.54, 0.03, .385]) 
cbax2 = fig.add_axes([0.91, 0.125, 0.03, .385]) 
cbar1 = fig.colorbar(cplot1,cax=cbax1,shrink=.5,panchor=(0,.75))
cbar2 = fig.colorbar(cplot2,cax=cbax2,shrink=.5,anchor=(0,.25))
cbar1.ax.set_ylabel('$\\zeta^-$',rotation=0,fontsize=15,labelpad=20)
cbar2.ax.set_ylabel('$\\zeta^+$',rotation=0,fontsize=15,labelpad=20)
cbar1.outline.set_visible(False)
cbar2.outline.set_visible(False)
plt.savefig('zeta-cartesian-both.pdf',bbox_inches='tight')

# Remapped simulations

## Geometric quantities

In [ ]:
# xi = (t,x,z)
# ξi = (τ,ξ,ζ)
J, K, g, G, Kdet = {}, {}, {}, {},{}
# Jij = J^j_i = d_xi(ξj)
J[1,0,0] = '1'
J[1,0,2] = '-(1-z)*ht/(2-h)'
J[1,1,1] = '1'
J[1,1,2] = '-(1-z)*hx/(2-h)'
J[1,2,2] = '1/(2-h)'
J[2,0,0] = '1'
J[2,0,2] = '-z*ht/h'
J[2,1,1] = '1'
J[2,1,2] = '-z*hx/h'
J[2,2,2] = '1/h'
# Kij = d_ξi(xj)
K[1,0,0] = '1'
K[1,0,2] = '(1-z)*ht'
K[1,1,1] = '1'
K[1,1,2] = '(1-z)*hx'
K[1,2,2] = '2-h'
K[2,0,0] = '1'
K[2,0,2] = 'z*ht'
K[2,1,1] = '1'
K[2,1,2] = 'z*hx'
K[2,2,2] = 'h'
# Kdet
Kdet[1] = '2-h'
Kdet[2] = 'h'
# gij = ωi.ωj
g[1,1,1] = '1'
g[1,1,2] = '-(1-z)*hx/(2-h)'
g[1,2,1] = 'g112'
g[1,2,2] = '(1 + ((1-z)*hx)**2)/(2-h)**2'
g[2,1,1] = '1'
g[2,1,2] = '-z*hx/h'
g[2,2,1] = 'g212'
g[2,2,2] = '(1 + (z*hx)**2)/h**2'
# Gi_jk = ωi.d_ξk(uj)
G[1,2,1,0] = '(1-z)*dx(ht)/(2-h)'
G[1,2,2,0] = '-ht/(2-h)'
G[1,2,1,1] = '(1-z)*dx(hx)/(2-h)'
G[1,2,1,2] = '-hx/(2-h)'
G[1,2,2,1] = 'G12_12'
G[2,2,1,0] = 'z*dx(ht)/h'
G[2,2,2,0] = 'ht/h'
G[2,2,1,1] = 'z*dx(hx)/h'
G[2,2,1,2] = 'hx/h'
G[2,2,2,1] = 'G22_12'

eps = {}
eps[1,2] = '1'
eps[2,1] = '-1'
dims = range(1,3)

## Remapped sharp-interface equations

In [ ]:
ν = 1e-1
κ = 1e-1
γ = 1e-1
μ = 1e-1
M = .2
N = 0
L = 1
nx = 256
nz = 128
dt = 1e-3
timestepper = 'SBDF2'
simname = f'salty-boussinesq-melting-tangent-example'
flt.makedir(f'{savedir}/frames/{simname}')
tend = 10
save_step = .01
save_freq = round(save_step/dt)

xbasis = de.Fourier('x',nx,interval=(0,4),dealias=3/2)
zbasis = de.Chebyshev('z',nz,interval=(0,1),dealias=3/2)
domain = de.Domain([xbasis,zbasis],grid_dtype=np.float64)
flt.save_domain(f'{savedir}/domain-{simname}.h5',domain)
x, z = domain.grids(scales=domain.dealias)
xx,zz = x+0*z, 0*x+z
dims = range(1,3)

problem = de.IVP(domain,variables=['u21','u22','p2','q2','T1','T1_2','T2','T2_2','C2','C2_2','h','ht','E','S'])
problem.meta[:]['z']['dirichlet'] = True
problem.meta['h','ht']['z']['constant'] = True
problem.meta['E','S']['x','z']['constant'] = True
problem.parameters['κ'] = κ
problem.parameters['ν'] = ν
problem.parameters['γ'] = γ
problem.parameters['μ'] = μ
problem.parameters['M'] = M
problem.parameters['N'] = N
problem.parameters['π'] = np.pi
problem.parameters['L'] = L
As = [4]*5
for i in range(len(As)): problem.parameters[f'A{i+1}'] = As[i]

problem.substitutions['d_1(A)'] = 'dx(A)'
problem.substitutions['d_2(A)'] = 'dz(A)'
problem.substitutions['T1_1'] = 'd_1(T1)'
problem.substitutions['T2_1'] = 'd_1(T2)'
problem.substitutions['C2_1'] = 'd_1(C2)'
problem.substitutions['hx'] = 'd_1(h)'
problem.substitutions['angle'] = 'sqrt(1 + hx**2)'
problem.substitutions['omz1'] = 'sqrt(1 + (hx)**2)/(2-h)'
problem.substitutions['omz2'] = 'sqrt(1 + (hx)**2)/h'
problem.substitutions['curvature'] = 'dx(hx)/angle**3'
problem.substitutions['xc'] = 'x'
problem.substitutions['zc1'] = 'h + (2-h)*z'
problem.substitutions['zc2'] = 'h*z'
for l in [1,2]:
    problem.substitutions[f'Kdet{l}'] = Kdet[l]
    for i in range(3):
        for j in range(3):
            problem.substitutions[f'J{l}{i}{j}'] = J.get((l,i,j),'0')
            problem.substitutions[f'K{l}{i}{j}'] = K.get((l,i,j),'0')
            problem.substitutions[f'g{l}{i}{j}'] = g.get((l,i,j),'0')
            for k in range(3):
                problem.substitutions[f'G{l}{i}_{j}{k}'] = G.get((l,i,j,k),'0')   
l = 2
for i in dims:
    for j in dims:
        problem.substitutions[f'cd_{i}_u{l}{j}'] = f'd_{i}(u{l}{j})' + ' + '.join(['']+[f'G{l}{j}_{k}{i}*u{l}{k}' for k in dims if G.get((l,j,k,i),0)])
problem.substitutions[f'div{l}'] = f'cd_1_u{l}1 + cd_2_u{l}2'
problem.substitutions[f'vorticity{l}'] = f'Kdet{l}*'+'({})'.format(' + '.join([f'{eps[i,j]}*g{l}{i}{k}*cd_{k}_u{l}{j}' for k in dims for j in dims for i in dims if eps.get((i,j),0)]))
for j in dims: problem.substitutions[f'curl_vorticity{l}{j}'] = f'(1/Kdet{l})*'+'({})'.format(' + '.join(f'{eps[i,j]}*d_{i}(q{l})' for i in dims if eps.get((i,j),0)))
for k in dims: problem.substitutions[f'q{l}_cross_u{l}{k}'] = f'Kdet{l}*q{l}*'+'({})'.format(' + '.join(f'{eps[i,j]}*g{l}{i}{k}*u{l}{j}' for i in dims for j in dims if eps.get((i,j),0)))
for j in dims: problem.substitutions[f'grad_p{l}{j}'] = ' + '.join([f'g{l}{i}{j}*d_{i}(p{l})' for i in dims])
for j in dims: problem.substitutions[f'f{l}{j}'] = f'(T{l} + N*C{l})*J{l}2{j}'
problem.substitutions['d_0(A)'] = '0'
for j in dims: problem.substitutions[f'adv_u{l}{j}'] = ' + '.join([f'J{l}0{i}*d_{i}(u{l}{j})' for i in range(3) if J.get((l,0,i),0)] + [f'J{l}0{i}*u{l}{k}*G{l}{j}_{k}{i}' for i in range(3) for k in dims if J.get((l,0,i),0) and G.get((l,j,k,i))])
for i in dims: problem.substitutions[f'dτu{l}{i}'] = f'- adv_u{l}{i} + ν*curl_vorticity{l}{i} - grad_p{l}{i} + q{l}_cross_u{l}{i} + f{l}{i}'
for i in dims: problem.substitutions[f'dtu{l}{i}'] = f'dτu{l}{i} + adv_u{l}{i}'
for l in dims: problem.substitutions[f'lapT{l}'] = '{} - ({})'.format(' + '.join([f'g{l}{i}{j}*d_{i}(T{l}_{j})' for i in dims for j in dims]),' + '.join([f'g{l}{i}{j}*T{l}_{k}*G{l}{k}_{i}{j}' for i in dims for j in dims for k in dims if G.get((l,k,i,j))]))
problem.substitutions[f'lapC{l}'] = '{} - ({})'.format(' + '.join([f'g{l}{i}{j}*d_{i}(C{l}_{j})' for i in dims for j in dims]),' + '.join([f'g{l}{i}{j}*C{l}_{k}*G{l}{k}_{i}{j}' for i in dims for j in dims for k in dims if G.get((l,k,i,j))]))
problem.substitutions[f'udotT1'] = '0'
problem.substitutions[f'udotT2'] = 'u21*T2_1 + u22*T2_2'
problem.substitutions[f'udotC2'] = 'u21*C2_1 + u22*C2_2'
problem.substitutions['ndotT1'] = 'left(( g121*T1_1 + g122*T1_2)/omz1)'
problem.substitutions['ndotT2'] = 'right((g221*T2_1 + g222*T2_2)/omz2)'
problem.substitutions['ndotC2'] = 'right((g221*C2_1 + g222*C2_2)/omz2)'
for l in dims: problem.substitutions[f'dτT{l}'] = 'κ*({}) - ({}) - ({})'.format(f'lapT{l}',f'udotT{l}',' + '.join([f'J{l}0{i}*T{l}_{i}' for i in dims if J.get((l,0,i))]))
problem.substitutions[f'dτC{l}'] = 'μ*({}) - ({}) - ({})'.format(f'lapC{l}',f'udotC{l}',' + '.join([f'J{l}0{i}*C{l}_{i}' for i in dims if J.get((l,0,i))]))
                                                     
# Cartesian quantities
l = 2
problem.substitutions[f'ux{l}'] = ' + '.join(f'u{l}{j}*K{l}{j}1' for j in dims if K.get((l,j,1)))
problem.substitutions[f'uz{l}'] = ' + '.join(f'u{l}{j}*K{l}{j}2' for j in dims if K.get((l,j,2)))
problem.substitutions[f'kenergy{l}'] = f'0.5*(ux{l}**2 + uz{l}**2)'
problem.substitutions[f'pr{l}'] = f'p{l} - kenergy{l}'
problem.substitutions[f'dxc{l}(A)'] = ' + '.join(f'J{l}1{j}*d_{j}(A)' for j in dims if J.get((l,1,j)))
problem.substitutions[f'dzc{l}(A)'] = ' + '.join(f'J{l}2{j}*d_{j}(A)' for j in dims if J.get((l,2,j)))
problem.substitutions[f'dtux{l}'] = f'- dxc{l}(p{l}) - ν*dzc{l}(q{l}) + q{l}*uz{l}'
problem.substitutions[f'dtuz{l}'] = f'- dzc{l}(p{l}) + ν*dxc{l}(q{l}) - q{l}*ux{l}'
# problem.substitutions[f'dtT1'] = f'- dzc{l}(p{l}) + ν*dxc{l}(q{l}) - q{l}*ux{l}'
# problem.substitutions[f'dtT2'] = f'- dzc{l}(p{l}) + ν*dxc{l}(q{l}) - q{l}*ux{l}'

problem.add_equation(f'       A1*(d_1(u{l}1) + d_2(u{l}2)) = A1*(d_1(u{l}1) + d_2(u{l}2)) - div{l}')
problem.add_equation(f'q{l} + A2*(d_2(u{l}1) - d_1(u{l}2)) = A2*(d_2(u{l}1) - d_1(u{l}2)) + vorticity{l}')
problem.add_equation(f'dt(u{l}1) + A3*d_1(p{l}) + A4*ν*d_2(q{l})             = A3*d_1(p{l}) + A4*ν*d_2(q{l})             + dτu{l}1')
problem.add_equation(f'dt(u{l}2) + A3*d_2(p{l}) - A4*ν*d_1(q{l}) - (T2+N*C2) = A3*d_2(p{l}) - A4*ν*d_1(q{l}) - (T2+N*C2) + dτu{l}2')    
problem.add_equation('T1_2 - d_2(T1) = 0')
problem.add_equation('T2_2 - d_2(T2) = 0')
problem.add_equation('dt(T1) - κ*A5*(d_1(T1_1) + d_2(T1_2)) = - κ*A5*(d_1(T1_1) + d_2(T1_2)) + dτT1')
problem.add_equation('dt(T2) - κ*A5*(d_1(T2_1) + d_2(T2_2)) = - κ*A5*(d_1(T2_1) + d_2(T2_2)) + dτT2')
problem.add_equation('dt(C2) - μ*A5*(d_1(C2_1) + d_2(C2_2)) = - μ*A5*(d_1(C2_1) + d_2(C2_2)) + dτC2')
problem.add_equation('C2_2 - d_2(C2) = 0')
problem.add_equation('dt(h) - ht = 0')
problem.add_equation('right(T2_2) - left(T1_2) + right(ht)*(L/κ) = right(T2_2) - left(T1_2) + right(ht)*(L/κ) - (ndotT2 - ndotT1 + right(ht/angle)*(L/κ))')
problem.add_equation('E = integ(Kdet1*T1 + Kdet2*(T2+1))')
problem.add_equation('S = integ(Kdet2*C2)')

# problem.add_bc("right(T1) = -1")
problem.add_bc("right(T1_2) = 0")
problem.add_bc('right(u21) = 0')
problem.add_bc('right(u22) = 0',condition='nx != 0')
problem.add_bc("right(T2 + M*C2 + γ*dx(hx)) = γ*(dx(hx) - curvature)") # Gibbs Thomson condition
problem.add_bc("right(T2) - left(T1) = 0")
problem.add_bc("right(C2_2) = - right((g221*C2_1 + (ht/h)*C2/μ)/g222)")
# problem.add_bc("right(C2_2 + C2_1 + ht) = right(C2_2 + C2_1 + ht) - right(g222*C2_2 + g221*C2_1 + (ht/h)*C2/μ)")
problem.add_bc('left(p2) = 0',condition='nx == 0')
problem.add_bc('left(u21) = 0')
problem.add_bc('left(u22) = 0')
# problem.add_bc("left(T2) = 1")
problem.add_bc("left(T2_2) = 0")
problem.add_bc("left(C2_2) = 0")

solver = problem.build_solver(eval(f'de.timesteppers.{timestepper}'))

solver.stop_sim_time = tend

u21,u22,p2,q2,T1,T1_2,T2,T2_2,C2,C2_2,h,ht,E,S = [solver.state[fname] for fname in problem.variables]
for field in u21,u22,p2,q2,T1,T1_2,T2,T2_2,C2,C2_2,h,ht,E,S:
    field.set_scales(domain.dealias)

h['g'] = 1
xc = x + 0*z
zc1 = h['g'] + (2-h['g'])*z
zc2 = h['g']*z
T1['g'] = 1-zc1
T2['g'] = 1-zc2 + np.exp(-((xc-2)**2+(zc2-.5)**2)*5**2)#1-z
T1.differentiate('z',out=T1_2)
T2.differentiate('z',out=T2_2)
C2['g'] = 0#.05 + (1-.05)*.5*(1-np.tanh(10*(zc2-.5)))
E['g'] = (h*(T2+1) + (2-h)*T1).evaluate().integrate()['g']
S['g'] = (h*C2).evaluate().integrate()['g']
v['g'] = 0

analysis = solver.evaluator.add_file_handler(f'{savedir}/analysis-{simname}',iter=save_freq, max_writes=100,mode='overwrite')
for task in problem.variables + ['zc1','zc2']: analysis.add_task(task)
interface = solver.evaluator.add_file_handler(f'{savedir}/interface-{simname}',iter=int(round(.01/dt)), max_writes=2001,mode='overwrite')
for task in ['h','ht','E','S']: interface.add_task(task)
for l in '2':
    for name in ['div','vorticity','zc','ux','uz','pr','kenergy','dtux','dtuz']:
        analysis.add_task(name+l)
    for i in dims:
        for task in [f'adv_u{l}{i}',f'curl_vorticity{l}{i}',f'grad_p{l}{i}',f'q{l}_cross_u{l}{i}',f'f{l}{i}']:
            analysis.add_task(task)        

start_time = time.time()
while solver.ok:
    if solver.iteration % 10 == 0: 
        print('{} time {:.1f} E {} S {:}'.format(solver.iteration, (start_time-time.time())/60,E['g'][0,0], S['g'][0,0]))
        if np.any(np.isnan(T2['g'])): 
            print('Broken')
            break
    solver.step(dt)
solver.step(dt)

## Remapped phase-field equations

In [ ]:
ν = 1e-2
κ = 1e-2
γ = 1e-2
μ = 1e-2
M = .2
N = 0
L = 1
β = 4/2.6482282
ϵ = np.sqrt(2e-5)
η = (β*ϵ)**2/ν
α = (5/6)*(L/κ)*ϵ

nx = 128
nz = 128
dt = 2.5e-4
timestepper = 'SBDF2'
simname = f'salty-boussinesq-melting-vpf-tangent-{ϵ:.0e}-example'
flt.makedir(f'{savedir}/frames/{simname}')
tend = 10
save_step = .1
save_freq = round(save_step/dt)

xbasis = de.Fourier('x',nx,interval=(0,4),dealias=1)
zb0 = de.Chebyshev('z0',32,interval=(0,10*ϵ),dealias=1)
zb1 = de.Chebyshev('z1',64,interval=(10*ϵ,1-10*ϵ),dealias=1)
zb2 = de.Chebyshev('z2',32,interval=(1-10*ϵ,1),dealias=1)
zbasis = de.Compound('z',[zb0,zb1,zb2])
domain = de.Domain([xbasis,zbasis],grid_dtype=np.float64)
flt.save_domain(f'{savedir}/domain-{simname}.h5',domain)
x, z = domain.grids(scales=domain.dealias)
xx,zz = x+0*z, 0*x+z
xc = xx

dims = range(1,3)

# Define GeneralFunction subclass to handle parities
GeneralFunction = de.operators.GeneralFunction
class HorizontalFunction(GeneralFunction):
    def __init__(self, domain, layout, func=None, args=[], kw={}, out=None,):
        super().__init__(domain, layout, func=func, args=args, kw=kw, out=out,)
    
    def meta_constant(self, axis):
        if axis == 1 or axis == 'z': return True
        else: return False

refname = 'salty-boussinesq-melting-tangent-conserved-passive'
post.merge_analysis(f'{savedir}/interface-{refname}',cleanup=True)
reffile = glob.glob(f'{savedir}/interface-{refname}/*.h5')[0]
t0s, x0s = flt.load_data(reffile,'sim_time','x/1.0',group='scales')
h0s, ht0s = flt.load_data(reffile,'h','ht',group='tasks')
from scipy.interpolate import RectBivariateSpline
h0s, ht0s = np.squeeze(h0s), np.squeeze(ht0s)
hspline = RectBivariateSpline(t0s,x0s,h0s)
htspline = RectBivariateSpline(t0s,x0s,ht0s)

def h_func(*args): return hspline(args[0].value,args[1].data[:,0]).T
def ht_func(*args): return htspline(args[0].value,args[1].data[:,0]).T
def h_op(*args,domain=domain,h_func=h_func): return HorizontalFunction(domain,layout='g',func=h_func,args=args)
def ht_op(*args,domain=domain,ht_func=ht_func): return HorizontalFunction(domain,layout='g',func=ht_func,args=args)
de.operators.parseables['h_op'] = h_op
de.operators.parseables['ht_op'] = ht_op

problem = de.IVP(domain,variables=['u11','u12','u21','u22','p1','q1','p2','q2','T1','T1_2','T2','T2_2','C1','C1_2','C2','C2_2','f1','f1_2','f2','f2_2','E','S'])

problem.meta[:]['z']['dirichlet'] = True
problem.meta['E','S']['x','z']['constant'] = True
problem.parameters['ν'] = ν
problem.parameters['κ'] = κ
problem.parameters['γ'] = γ
problem.parameters['μ'] = μ
problem.parameters['M'] = M
problem.parameters['N'] = N
problem.parameters['δ'] = 1e-4
problem.parameters['ε'] = ϵ
problem.parameters['L'] = L
problem.parameters['η'] = η
problem.parameters['α'] = α
As = [4]*5
for i in range(len(As)): problem.parameters[f'A{i+1}'] = As[i]

problem.substitutions['h'] = 'h_op(t,x)'
problem.substitutions['ht'] = 'ht_op(t,x)'
problem.substitutions['d_1(A)'] = 'dx(A)'
problem.substitutions['d_2(A)'] = 'dz(A)'
problem.substitutions['T1_1'] = 'd_1(T1)'
problem.substitutions['T2_1'] = 'd_1(T2)'
problem.substitutions['C1_1'] = 'd_1(C1)'
problem.substitutions['C2_1'] = 'd_1(C2)'
problem.substitutions['f1_1'] = 'd_1(f1)'
problem.substitutions['f2_1'] = 'd_1(f2)'
problem.substitutions['hx'] = 'd_1(h)'
problem.substitutions['angle'] = 'sqrt(1 + hx**2)'
problem.substitutions['omz1'] = 'sqrt(1 + (hx)**2)/(2-h)'
problem.substitutions['omz2'] = 'sqrt(1 + (hx)**2)/h'
problem.substitutions['curvature'] = 'dx(hx)/angle**3'
problem.substitutions['xc'] = 'x'
problem.substitutions['zc1'] = 'h + (2-h)*z'
problem.substitutions['zc2'] = 'h*z'
problem.substitutions['d_0(A)'] = '0'
for l in [1,2]:
    problem.substitutions[f'Kdet{l}'] = Kdet[l]
    for i in range(3):
        for j in range(3):
            problem.substitutions[f'J{l}{i}{j}'] = J.get((l,i,j),'0')
            problem.substitutions[f'K{l}{i}{j}'] = K.get((l,i,j),'0')
            problem.substitutions[f'g{l}{i}{j}'] = g.get((l,i,j),'0')
            for k in range(3):
                problem.substitutions[f'G{l}{i}_{j}{k}'] = G.get((l,i,j,k),'0')   

    for i in dims:
        for j in dims:
            problem.substitutions[f'cd_{i}_u{l}{j}'] = f'd_{i}(u{l}{j})' + ' + '.join(['']+[f'G{l}{j}_{k}{i}*u{l}{k}' for k in dims if G.get((l,j,k,i),0)])
    problem.substitutions[f'div{l}'] = f'cd_1_u{l}1 + cd_2_u{l}2'
    problem.substitutions[f'vorticity{l}'] = f'Kdet{l}*'+'({})'.format(' + '.join([f'{eps[i,j]}*g{l}{i}{k}*cd_{k}_u{l}{j}' for k in dims for j in dims for i in dims if eps.get((i,j),0)]))
    for j in dims: problem.substitutions[f'curl_vorticity{l}{j}'] = f'(1/Kdet{l})*'+'({})'.format(' + '.join(f'{eps[i,j]}*d_{i}(q{l})' for i in dims if eps.get((i,j),0)))
    for k in dims: problem.substitutions[f'q_cross_u{l}{k}'] = f'Kdet{l}*q{l}*'+'({})'.format(' + '.join(f'{eps[i,j]}*g{l}{i}{k}*u{l}{j}' for i in dims for j in dims if eps.get((i,j),0)))
    for j in dims: problem.substitutions[f'grad_p{l}{j}'] = ' + '.join([f'g{l}{i}{j}*d_{i}(p{l})' for i in dims])
    for j in dims: problem.substitutions[f'f{l}{j}'] = f'(T{l}+N*C{l})*J{l}2{j}'
    for j in dims: problem.substitutions[f'adv_u{l}{j}'] = ' + '.join([f'J{l}0{i}*d_{i}(u{l}{j})' for i in dims if J.get((l,0,i),0)] + [f'J{l}0{i}*u{l}{k}*G{l}{j}_{k}{i}' for i in range(3) for k in dims if J.get((l,0,i),0) and G.get((l,j,k,i))])
    for i in dims: problem.substitutions[f'dτu{l}{i}'] = f'- adv_u{l}{i} + ν*curl_vorticity{l}{i} - grad_p{l}{i} + q_cross_u{l}{i} - (f{l}/η)*u{l}{i} + f{l}{i}'
    for i in dims: problem.substitutions[f'dtu{l}{i}'] = f'dτu{l}{i} + adv_u{l}{i}'
    problem.substitutions[f'lapT{l}'] = '{} - ({})'.format(' + '.join([f'g{l}{i}{j}*d_{i}(T{l}_{j})' for i in dims for j in dims]),' + '.join([f'g{l}{i}{j}*T{l}_{k}*G{l}{k}_{i}{j}' for i in dims for j in dims for k in dims if G.get((l,k,i,j))]))
    problem.substitutions[f'lapC{l}'] = '{} - ({})'.format(' + '.join([f'g{l}{i}{j}*d_{i}(C{l}_{j})' for i in dims for j in dims]),' + '.join([f'g{l}{i}{j}*C{l}_{k}*G{l}{k}_{i}{j}' for i in dims for j in dims for k in dims if G.get((l,k,i,j))]))
    problem.substitutions[f'lapf{l}'] = '{} - ({})'.format(' + '.join([f'g{l}{i}{j}*d_{i}(f{l}_{j})' for i in dims for j in dims]),' + '.join([f'g{l}{i}{j}*f{l}_{k}*G{l}{k}_{i}{j}' for i in dims for j in dims for k in dims if G.get((l,k,i,j))]))
    problem.substitutions[f'udotT{l}'] = f'u{l}1*T{l}_1 + u{l}2*T{l}_2'
    problem.substitutions[f'udotC{l}'] = f'u{l}1*C{l}_1 + u{l}2*C{l}_2'
    problem.substitutions[f'ndotT{l}'] = f'left(( g{l}21*T{l}_1 + g{l}22*T{l}_2)/omz{l})'
    problem.substitutions[f'ndotf{l}'] = f'left(( g{l}21*f{l}_1 + g{l}22*f{l}_2)/omz{l})'
    problem.substitutions[f'f_flux{l}'] = f'-ε**(-2)*f{l}*(1-f{l})*(γ*(1-2*f{l}) + ε*(T{l}+M*C{l}))'
    problem.substitutions[f'dtf{l}'] = f'(1/α)*(γ*lapf{l} + f_flux{l})'
    problem.substitutions[f'dτf{l}'] =  f'dtf{l} - '+'({})'.format(' + '.join([f'J{l}0{i}*f{l}_{i}' for i in dims if J.get((l,0,i))]))
    problem.substitutions[f'gradfdotgradC{l}'] = ' + '.join([f'g{l}{i}{j}*f{l}_{i}*C{l}_{j}' for i in dims for j in dims])
    problem.substitutions[f'dτT{l}'] = 'κ*({}) - ({}) + ({}) - ({})'.format(f'lapT{l}',f'udotT{l}',f'L*dtf{l}', ' + '.join([f'J{l}0{i}*T{l}_{i}' for i in dims if J.get((l,0,i))]))
    problem.substitutions[f'dτC{l}'] = 'μ*({}) - ({}) + ({}) - ({})'.format(f'lapC{l}',f'udotC{l}',f'(dtf{l}*C{l} - μ*gradfdotgradC{l})/(1-f{l}+δ)', ' + '.join([f'J{l}0{i}*C{l}_{i}' for i in dims if J.get((l,0,i))]))
# problem.substitutions['dτh'] = ' - (1/2)*(left(dtf1/(J122*f1_2)) + right(dtf2/(J222*f2_2)))'
# # Cartesian quantities
# problem.substitutions[f'ux{l}'] = ' + '.join(f'u{l}{j}*K{l}{j}1' for j in dims if K.get((l,j,1)))
# problem.substitutions[f'uz{l}'] = ' + '.join(f'u{l}{j}*K{l}{j}2' for j in dims if K.get((l,j,2)))
# problem.substitutions[f'kenergy{l}'] = f'0.5*(ux{l}**2 + uz{l}**2)'
# problem.substitutions[f'pr{l}'] = f'p{l} - kenergy{l}'
# problem.substitutions[f'dxc{l}(A)'] = ' + '.join(f'J{l}1{j}*d_{j}(A)' for j in dims if J.get((l,1,j)))
# problem.substitutions[f'dzc{l}(A)'] = ' + '.join(f'J{l}2{j}*d_{j}(A)' for j in dims if J.get((l,2,j)))
# problem.substitutions[f'dtux{l}'] = f'- dxc{l}(p{l}) - ν*dzc{l}(q{l}) + q{l}*uz{l}'
# problem.substitutions[f'dtuz{l}'] = f'- dzc{l}(p{l}) + ν*dxc{l}(q{l}) - q{l}*ux{l}'
# problem.substitutions[f'dtT1'] = f'- dzc{l}(p{l}) + ν*dxc{l}(q{l}) - q{l}*ux{l}'
for l in [1,2]:
    problem.add_equation(f'       A1*(d_1(u{l}1) + d_2(u{l}2)) = A1*(d_1(u{l}1) + d_2(u{l}2)) - div{l}')
    problem.add_equation(f'q{l} + A2*(d_2(u{l}1) - d_1(u{l}2)) = A2*(d_2(u{l}1) - d_1(u{l}2)) + vorticity{l}')
    problem.add_equation(f'dt(u{l}1) + A3*d_1(p{l}) + A4*ν*d_2(q{l}) =             A3*d_1(p{l}) + A4*ν*d_2(q{l}) + dτu{l}1')
    problem.add_equation(f'dt(u{l}2) + A3*d_2(p{l}) - A4*ν*d_1(q{l}) - T2 = - T2 + A3*d_2(p{l}) - A4*ν*d_1(q{l}) + dτu{l}2')    
    problem.add_equation(f'T{l}_2 - d_2(T{l}) = 0')
    problem.add_equation(f'dt(T{l}) - κ*A5*(d_1(T{l}_1) + d_2(T{l}_2)) = - κ*A5*(d_1(T{l}_1) + d_2(T{l}_2)) + dτT{l}')
    problem.add_equation(f'C{l}_2 - d_2(C{l}) = 0')
    problem.add_equation(f'dt(C{l}) - μ*A5*(d_1(C{l}_1) + d_2(C{l}_2)) = - μ*A5*(d_1(C{l}_1) + d_2(C{l}_2)) + dτC{l}')
    problem.add_equation(f'f{l}_2 - d_2(f{l}) = 0')
    problem.add_equation(f'α*dt(f{l})-γ*A5*(d_1(f{l}_1) + d_2(f{l}_2)) = - γ*A5*(d_1(f{l}_1) + d_2(f{l}_2)) + α*dτf{l}')
# problem.add_equation('ht - (κ/L)*(right(T2_2) - right(T1_2)) = - (κ/L)*(right(T2_2) - right(T1_2)) + dτh')
# problem.add_equation('ht = -100*(left(f1) - .5)')
# problem.add_equation('dt(h) - ht = 0')
problem.add_equation('E = integ(Kdet1*(T1 + L*(1-f1)) + Kdet2*(T2 + L*(1-f2)))')
problem.add_equation('S = integ(Kdet1*(1-f1)*C1 + Kdet2*(1-f2)*C2)')

problem.add_bc('right(u11) = 0')
problem.add_bc('right(u12) = 0',condition='nx != 0')
# problem.add_bc('right(T1) = -1')
problem.add_bc('right(T1_2) = 0')
problem.add_bc('right(C1_2) = 0')
problem.add_bc('right(f1_2) = 0')
problem.add_bc('left(u11) - right(u21) = 0')
problem.add_bc('left(u12) - right(u22) = left((h-1)*u12) + right((h-1)*u22)')
problem.add_bc('left(p1) - right(p2) = 0')
problem.add_bc('left(q1) - right(q2) = 0')
problem.add_bc('right(T2) - left(T1) = 0')
problem.add_bc('right(C2) - left(C1) = 0')
problem.add_bc('right(f2) - left(f1) = 0')
# problem.add_bc('right(f2) = 0.5')
problem.add_bc('left(T1_2) - right(T2_2) = left((1-h)*T1_2) + right((1-h)*T2_2)')
problem.add_bc('left(C1_2) - right(C2_2) = left((1-h)*C1_2) + right((1-h)*C2_2)')
problem.add_bc('left(f1_2) - right(f2_2) = left((1-h)*f1_2) + right((1-h)*f2_2)')
problem.add_bc('left(p2) = 0',condition='nx == 0')
problem.add_bc('left(u21) = 0')
problem.add_bc('left(u22) = 0')
problem.add_bc('left(T2_2) = 0')
# problem.add_bc('left(T2) = 1')
problem.add_bc('left(C2_2) = 0')
problem.add_bc('left(f2_2) = 0')

solver = problem.build_solver(eval(f'de.timesteppers.{timestepper}'))

solver.stop_sim_time = tend

fields = {fname:solver.state[fname] for fname in problem.variables}
for fname,field in fields.items():
    field.set_scales(domain.dealias)
hop, htop = solver.evaluator.vars['h'], solver.evaluator.vars['ht']
h, ht = hop.evaluate(), htop.evaluate()
T1, T2, C1,C2,f1, f2,E,S = [fields[name] for name in ['T1','T2','C1','C2','f1','f2','E','S']]

xc = xx
zc1 = h['g'] + (2-h['g'])*zz
zc2 = h['g']*zz
kx,kz = domain.elements(0),domain.elements(1)
kxx,kzz = kx+0*kz, 0*kx + kz
kz2 = kzz
kz1 = kzz + kzz.max()

# initial conditions
h['g'] = 1
T1['g'] = 1-zc1
T2['g'] = 1-zc2 + np.exp(-((xc-2)**2+(zc2-.5)**2)*5**2)
T1.differentiate('z',out=fields['T1_2'])
T2.differentiate('z',out=fields['T2_2'])
C1['g'] = 0.05 + (1-0.05)*.5*(1 - np.tanh(10*(zc1-.5)))
C2['g'] = 0.05 + (1-0.05)*.5*(1 - np.tanh(10*(zc2-.5)))
C1.differentiate('z',out=fields['C1_2'])
C2.differentiate('z',out=fields['C2_2'])
zc1 = h['g'] + (2-h['g'])*zz
zc2 = h['g']*zz
f1['g'] = .5*(1+np.tanh((1/(2*ϵ))*(zc1-h['g'])/np.sqrt(1+d(h,'x')**2)['g']))
f2['g'] = .5*(1+np.tanh((1/(2*ϵ))*(zc2-h['g'])/np.sqrt(1+d(h,'x')**2)['g']))
f1.differentiate('z',out=fields['f1_2'])
f2.differentiate('z',out=fields['f2_2'])
E['g'] = (h*(T2+L*(1-f2)) + (2-h)*(T1+L*(1-f1))).evaluate().integrate()['g']
S['g'] = (h*(1-f2)*C2 + (2-h)*(1-f1)*C1).evaluate().integrate()['g']

analysis = solver.evaluator.add_file_handler(f'{savedir}/analysis-{simname}',iter=save_freq, max_writes=100,)#mode='overwrite')
for task in problem.variables + ['h','ht','zc1','zc2']: analysis.add_task(task)
for l in '2':
    for name in ['div','vorticity','ux','uz','pr','kenergy','dtux','dtuz']:
        analysis.add_task(name+l)

while solver.ok:
    if solver.iteration % 100 == 0: 
        logger.info(f'It: {solver.iteration}, Sim time: {solver.sim_time:.3f}, Salt: {S["g"][0,0]:.6f}')
        if np.any(np.isnan(T2['g'])): break
    solver.step(dt)
solver.step(dt)

## Analysis